In [1]:
import pandas as pd
import pyranges as pr
import upsetplot
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [3]:
h5 = 'cerberus_annot_triplets.h5'
filt_ab = 'cerberus_filtered_abundance.tsv'
feat = 'tss'
obs_source = 'lapa'
ref_sources = ['v40', 'v29']
support_sources = ['encode_cage',
                   'fantom_cage',
                   'encode_rampage']                   
support_label = 'CAGE, RAMPAGE'
ref = '../../refs/cerberus/v40_transcript_metadata.tsv'